<br><br>
**INTRO**

In [1]:
import sys.process._

import sys.process._

In [2]:
"ls -l" !

total 104
-rwxrwxrwx 1 jovyan  1000  2083 Sep 12 17:33 about
drwxrwxrwx 4 jovyan  1000  4096 Sep 12 17:26 data
-rw-r--r-- 1 jovyan users  1005 Sep 14 01:15 div-id
drwxr-xr-x 2 jovyan users  4096 Sep 14 02:30 img
-rwxrwxrwx 1 jovyan  1000 86845 Sep 14 02:50 spark_sna_eda.ipynb


res1: Int = 0

In [3]:
def wd = {val dir = "pwd" !!; dir.stripLineEnd}

defined function wd

In [4]:
// 1. Скачайте датасет

val parquet_train = f"${wd}/data/SNAHackathon2019/collabTrain"
val parquet_test = f"${wd}/data/SNAHackathon2019/collabTest"

parquet_train: String = "/home/jovyan/work/HW002/spark_homework/data/SNAHackathon2019/collabTrain"
parquet_test: String = "/home/jovyan/work/HW002/spark_homework/data/SNAHackathon2019/collabTest"

In [5]:
f"du -s -h $parquet_train " !

2.2G	/home/jovyan/work/HW002/spark_homework/data/SNAHackathon2019/collabTrain


res4: Int = 0

In [6]:
f"du -s -h $parquet_test " !

126M	/home/jovyan/work/HW002/spark_homework/data/SNAHackathon2019/collabTest


res5: Int = 0

<br><br>
**IMPORT LIBRARY AND CREATE SPARK SESSION**

In [7]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

import spark.implicits._


Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  

import $ivy.$                                    


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@3b3c25e9
import spark.implicits._


In [7]:
// 2. Ознакомьтесь с его описанием

**LOAD DATA**
<br><br>
<font size="2.0">
instanceId_userId — идентификатор пользователя (анонимизированный);  
instanceId_objectType — тип объекта;  
instanceId_objectId — идентификатор объекта (анонимизированный);  
feedback — массив с типами реакций пользователя (наличие в массиве токена Liked говорит о том, что объект получил «класс» от пользователя);  
audit_clientType — тип платформы, с которой зашёл пользователь;  
audit_timestamp — время, когда строилась лента;  
metadata_ownerId — автор показанного объекта (анонимизированный);  
metadata_createdAt — дата создания показанного объекта.  
</font>
<br><font size="2.0">
audit_* — расширенная информация о контексте построения ленты;  
metadata_* — расширенная информация о самом объекте;  
userOwnerCounters_* — информация о предыдущих взаимодействиях пользователя и автора контента;  
ownerUserCounters_* — информация о предыдущих взаимодействиях автора контента и пользователя;  
membership_* — информация о членстве пользователя в группе, где опубликован контент;  
user_* — подробная информация о пользователе;  
auditweights_* — большое количество runtime-признаков, извлечённых текущей системой.  
</font>

In [8]:
// load from parquet
val df = spark.read.parquet(parquet_train).cache()

// get dimention of dataframe
val shape = (df.count().toInt, df.columns.length)

// group attribute by prefix name
def colName = df.columns.map(col(_)).groupBy(_.toString.split("_")(0))

// group attribute by spark type
def colType = df.schema.fields.map(f => (col(f.name), f.dataType.toString)).groupBy(_._2).mapValues(x => x.map(y => y._1))

parquet at cmd7.sc:1

48 / 48

parquet at cmd7.sc:1

1 / 1

count at cmd7.sc:4

28 / 28

count at cmd7.sc:4

1 / 1

df: Dataset[Row] = [instanceId_userId: int, instanceId_objectType: string ... 167 more fields]
shape: (Int, Int) = (18286575, 169)
defined function colName
defined function colType

In [9]:
df.printSchema

root
 |-- instanceId_userId: integer (nullable = true)
 |-- instanceId_objectType: string (nullable = true)
 |-- instanceId_objectId: integer (nullable = true)
 |-- audit_pos: long (nullable = true)
 |-- audit_clientType: string (nullable = true)
 |-- audit_timestamp: long (nullable = true)
 |-- audit_timePassed: long (nullable = true)
 |-- audit_experiment: string (nullable = true)
 |-- audit_resourceType: long (nullable = true)
 |-- metadata_ownerId: integer (nullable = true)
 |-- metadata_ownerType: string (nullable = true)
 |-- metadata_createdAt: long (nullable = true)
 |-- metadata_authorId: integer (nullable = true)
 |-- metadata_applicationId: long (nullable = true)
 |-- metadata_numCompanions: integer (nullable = true)
 |-- metadata_numPhotos: integer (nullable = true)
 |-- metadata_numPolls: integer (nullable = true)
 |-- metadata_numSymbols: integer (nullable = true)
 |-- metadata_numTokens: integer (nullable = true)
 |-- metadata_numVideos: integer (nullable = true)
 |-- me

 |-- auditweights_userOwner_USER_INTERNAL_LIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_INTERNAL_UNLIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_PRESENT_SEND: double (nullable = true)
 |-- auditweights_userOwner_USER_PROFILE_VIEW: double (nullable = true)
 |-- auditweights_userOwner_USER_SEND_MESSAGE: double (nullable = true)
 |-- auditweights_userOwner_USER_STATUS_COMMENT_CREATE: double (nullable = true)
 |-- auditweights_userOwner_VIDEO: double (nullable = true)
 |-- auditweights_userOwner_VOTE_POLL: double (nullable = true)
 |-- auditweights_x_ActorsRelations: long (nullable = true)
 |-- auditweights_likersSvd_spark_hyper: double (nullable = true)
 |-- auditweights_source_PROMO: double (nullable = true)
 |-- date: date (nullable = true)



In [10]:
colName.foreach {case (key, value) => println (f"$key: ${value.length}")}

instanceId: 3
membership: 4
relationsMask: 1
userOwnerCounters: 27
feedback: 1
date: 1
metadata: 14
objectId: 1
audit: 6
owner: 13
ownerUserCounters: 27
auditweights: 58
user: 13


In [11]:
df.select(colName("instanceId"): _*).show(3)

show at cmd10.sc:1

1 / 1

+-----------------+---------------------+-------------------+
|instanceId_userId|instanceId_objectType|instanceId_objectId|
+-----------------+---------------------+-------------------+
|              108|                 Post|           18452434|
|              231|                 Post|           31980032|
|              423|                 Post|           33834009|
+-----------------+---------------------+-------------------+
only showing top 3 rows



In [12]:
colType.foreach {case (key, value) => println (f"$key: ${value.length}")}

StringType: 6
LongType: 17
IntegerType: 32
ArrayType(StringType,true): 2
DoubleType: 111
DateType: 1


In [13]:
df.select(colType("StringType"): _*).show(3)

show at cmd12.sc:1

1 / 1

+---------------------+----------------+----------------+-------------------+-----------------+-----------------+
|instanceId_objectType|audit_clientType|audit_experiment| metadata_ownerType|metadata_platform|membership_status|
+---------------------+----------------+----------------+-------------------+-----------------+-----------------+
|                 Post|             MOB|    XPRM-5386_G1|GROUP_OPEN_OFFICIAL|            OTHER|                A|
|                 Post|             MOB|    XPRM-5386_G2|GROUP_OPEN_OFFICIAL|            OTHER|                A|
|                 Post|             API|    XPRM-5386_G9|         GROUP_OPEN|          ANDROID|             null|
+---------------------+----------------+----------------+-------------------+-----------------+-----------------+
only showing top 3 rows



In [14]:
df.select(colType("ArrayType(StringType,true)"): _*).show(3)

show at cmd13.sc:1

1 / 1

+--------------------+----------------+
|    metadata_options|        feedback|
+--------------------+----------------+
|[HAS_TEXT, HAS_PH...|         [Liked]|
|[HAS_TEXT, HAS_PH...|[Clicked, Liked]|
|[IS_PART_OF_TOPIC...|         [Liked]|
+--------------------+----------------+
only showing top 3 rows



In [14]:
// summary report : TODO

In [14]:
// 3. Постройте топ популярных групп на портале

In [15]:
// Посмотрим, что лежит в feedback

(
    df
    .select("feedback")
    .withColumn("feedback", explode($"feedback"))
    .groupBy("feedback")
    .count()
    .orderBy(desc("count"))
    .withColumn("fraction", round($"count"/ shape._1, 2))
    .show()
)


show at cmd14.sc:8

28 / 28

show at cmd14.sc:8

200 / 200

+---------+--------+--------+
| feedback|   count|fraction|
+---------+--------+--------+
|  Ignored|12421394|    0.68|
|    Liked| 3220429|    0.18|
|  Clicked| 2258630|    0.12|
| Disliked| 1665403|    0.09|
| ReShared|  321631|    0.02|
|   Viewed|  170031|    0.01|
|Commented|   36244|     0.0|
|  Unliked|   33428|     0.0|
|Complaint|     263|     0.0|
+---------+--------+--------+



In [16]:
// Рейтинг

val colSeq = Seq("date", "hour", "metadata_authorId", "instanceId_objectId", "instanceId_userId", "feedback")

val out = df

.withColumn("hour", hour(($"audit_timestamp"/1000).cast(types.TimestampType)) )
.select(colSeq.map(col(_)): _*)
.withColumnRenamed("metadata_authorId", "authorId")
.withColumnRenamed("instanceId_objectId", "objectId")
.withColumnRenamed("instanceId_userId", "userId")
.withColumn("Liked", ( array_contains($"feedback", "Liked")).cast(types.IntegerType) )
.withColumn("Disliked", ( array_contains($"feedback", "Disliked")).cast(types.IntegerType) )
.withColumn("Interacted", ( !array_contains($"feedback", "Ignored")).cast(types.IntegerType) )

println("SAMPLE")
out.show(10)


val out1 = out

.groupBy("authorId")
.agg(
    count("*").alias("baseAll"), 
    countDistinct("objectId").alias("objShare"),
    countDistinct("userId").alias("usrShare"),
    sum("Liked").alias("lik"),
    sum("Disliked").alias("dis"),
    sum("Interacted").alias("act")
)

// baseAll       - сколько всего было фактов комуникации от данного Автора
// objShare      - сколько уникальных объектов было показано
// usrShare      - сколько уникальных юзеров это увидело
// lik|dis|act   - результат коммуникации относительно baseAll

println("GROUP")
out1.show(10)

// расчитаем удельные промежуточные метрики

val cntAll = out.count()
val uniqObj = out.select($"objectId").groupBy("objectId").count().first().getInt(0)
val uniqUsr = out.select($"userId").groupBy("userId").count().first().getInt(0)
// val minBase = 100

val out2 = out1
// .filter($"baseAll" >= minBase)

.withColumn( "shareAll", round($"baseAll"*100 / cntAll, 4).cast(types.DoubleType) )
.withColumn( "shareObj", round($"objShare"*100 / uniqObj, 4).cast(types.DoubleType) )
.withColumn( "shareUsr", round($"usrShare"*100 / uniqUsr, 4).cast(types.DoubleType) )
.withColumn( "likesMore", round(($"lik" - $"dis")*100 / $"baseAll", 2) )
.withColumn( "interMore", round($"act"*100 / $"baseAll", 2) )

.select('authorId, 'shareAll, 'shareObj, 'shareUsr, 'likesMore, 'interMore)

println("PROXY")
out2.show(10)

def specWindow(colName: String) = org.apache.spark.sql.expressions.Window.orderBy(colName)

val out3 = out2
.withColumn("R1", rank().over(specWindow("shareAll")))
.withColumn("R2", rank().over(specWindow("shareObj")))
.withColumn("R3", rank().over(specWindow("shareUsr")))
.withColumn("R4", rank().over(specWindow("likesMore")))
.withColumn("R5", rank().over(specWindow("interMore")))

val out4 = out3
.withColumn("avgR", round(
    (
        out3("R1")*5+out3("R2")*10+out3("R3")*10+out3("R4")*50+out3("R5")*25
    )/500, 2)
            .cast(types.DoubleType))

.orderBy(desc("avgR"))

// formula from my head = 5 10 10 50 25

println("RANKING")
out4.show(10)




SAMPLE


show at cmd15.sc:15

1 / 1

+----------+----+--------+--------+------+----------------+-----+--------+----------+
|      date|hour|authorId|objectId|userId|        feedback|Liked|Disliked|Interacted|
+----------+----+--------+--------+------+----------------+-----+--------+----------+
|2018-03-04|  20|   73356|18452434|   108|         [Liked]|    1|       0|         1|
|2018-03-04|  21|  828613|31980032|   231|[Clicked, Liked]|    1|       0|         1|
|2018-03-04|  16| 1076820|33834009|   423|         [Liked]|    1|       0|         1|
|2018-03-04|   7|  608137|25653019|   624|       [Ignored]|    0|       0|         0|
|2018-03-04|  16|  565604|24024934|   768|       [Ignored]|    0|       0|         0|
|2018-03-04|  10|  981198|25075524|  1206|       [Ignored]|    0|       0|         0|
|2018-03-04|   7|  579806|18132300|  1752|[Clicked, Liked]|    1|       0|         1|
|2018-03-04|   7|  200111| 2402728|  1752|         [Liked]|    1|       0|         1|
|2018-03-04|  10|  980484|17153719|  1752|       [Igno

show at cmd15.sc:36

...

200 / 200

show at cmd15.sc:80

1 / 1

+--------+--------+--------+--------+---------+---------+------+------+------+------+------+--------+
|authorId|shareAll|shareObj|shareUsr|likesMore|interMore|    R1|    R2|    R3|    R4|    R5|    avgR|
+--------+--------+--------+--------+---------+---------+------+------+------+------+------+--------+
|  688041|  0.0056|  3.0E-4|  1.4281|    45.09|    50.69|295914|284790|296068|281493|244813|54966.25|
| 1051082|  0.0029|  3.0E-4|  0.7536|    47.01|    51.12|294229|284790|294581|281595|244897|54934.06|
| 1119893|  8.0E-4|  7.0E-4|  0.1947|    36.23|     55.8|287732|292956|288190|279315|247057|54784.59|
|  185688|   0.003|  3.0E-4|  0.7649|    39.02|    50.09|294348|284790|294623|279874|244767|54757.49|
|  618013|  0.0034|  2.0E-4|  0.8622|    34.09|    50.49|294735|277200|294962|279103|244784|54540.09|
|  388312|  0.0016|  2.0E-4|   0.398|    37.68|     50.7|291825|277200|292127|279766|244816|54522.19|
|  453953|  6.0E-4|  4.0E-4|  0.1298|    34.82|    50.89|285324|288355|285035|2791

colSeq: Seq[String] = List(
  "date",
  "hour",
  "metadata_authorId",
  "instanceId_objectId",
  "instanceId_userId",
  "feedback"
)
out: DataFrame = [date: date, hour: int ... 7 more fields]
out1: DataFrame = [authorId: int, baseAll: bigint ... 5 more fields]
cntAll: Long = 18286575L
uniqObj: Int = 12871510
uniqUsr: Int = 70863
out2: DataFrame = [authorId: int, shareAll: double ... 4 more fields]
defined function specWindow
out3: DataFrame = [authorId: int, shareAll: double ... 9 more fields]
out4: Dataset[Row] = [authorId: int, shareAll: double ... 10 more fields]

<br><br>
**IMPORT LIBRARY FOR VISUALIZE**

In [17]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.6`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


In [18]:
// 4. Построите гистограммы популярности/активности групп на портале по времени суток

val hist1 = (
    out
    .select('hour, 'Interacted)
    .groupBy('hour)
    .agg(sum('Interacted).alias("Interacted"))
    .orderBy('hour)
)
hist1.show(24)

show at cmd17.sc:8

28 / 28

show at cmd17.sc:8

200 / 200

+----+----------+
|hour|Interacted|
+----+----------+
|   0|     55733|
|   1|     76503|
|   2|    107668|
|   3|    157802|
|   4|    208885|
|   5|    251970|
|   6|    267698|
|   7|    272816|
|   8|    275905|
|   9|    279034|
|  10|    288987|
|  11|    299312|
|  12|    315781|
|  13|    336393|
|  14|    365712|
|  15|    395531|
|  16|    414272|
|  17|    410942|
|  18|    378458|
|  19|    290172|
|  20|    190048|
|  21|    109330|
|  22|     65414|
|  23|     50815|
+----+----------+



hist1: Dataset[Row] = [hour: int, Interacted: bigint]

### https://alexarchambault.github.io/plotly-scala/

In [19]:
val xValue : Array[Int] = new scala.Array(24)
val yValue : Array[Int] = new scala.Array(24)

hist1.collect().zipWithIndex.foreach{
    case(v, i) => {
        xValue(i) = v.getInt(0)
        yValue(i) = v.getLong(1).toInt/1000
    }
}

val trace1 = Bar(
  xValue.toVector,
  yValue.toVector,
//   text = Seq("label 1", "label 2", "label 3"),
  marker = Marker(
//     color = Color.RGB(158, 202, 225),
    opacity = 0.6,
    line = Line(
//       color = Color.RGB(8, 48, 107),
      width = 1.5
    )
  )
)

val data = Seq(trace1)

val annotations = xValue.zip(yValue).map {
  case (x, y) =>
    Annotation(
      x = x,
      y = y,
      text = y.toString,
      xanchor = Anchor.Center,
      yanchor = Anchor.Bottom,
      showarrow = false
    )
}

val layout = Layout(
  title = "(1k) Interaction (Not Ignored) x Time",
  annotations = annotations
)

plot(data, layout)


collect at cmd18.sc:4

28 / 28

collect at cmd18.sc:4

200 / 200

collect at cmd18.sc:4

200 / 200

collect at cmd18.sc:4

24 / 24

xValue: Array[Int] = Array(
  0,
...
yValue: Array[Int] = Array(
  55,
...
trace1: Bar = Bar(
  Doubles(
...
data: Seq[Bar] = List(
  Bar(
...
annotations: Array[Annotation] = Array(
  Annotation(
...
layout: Layout = Layout(
  Some("(1k) Interaction (Not Ignored) x Time"),
...
res18_7: String = "plot-711de1c1-faa2-471f-9314-a32520aaeb90"

In [34]:
// 5. Посчитайте корреляцию признаков с целевой переменной

val df1 = df.withColumn("target", (array_contains($"feedback", "Liked")).cast(types.IntegerType))

colType("LongType").foreach{
    x => println(x + ": " + df1.stat.corr("target", x.toString))
}


corr at cmd33.sc:4

28 / 28

corr at cmd33.sc:4

4 / 4

((audit_pos + : ) + 0.005763417569643012)


...

28 / 28

corr at cmd33.sc:4

4 / 4

((auditweights_x_ActorsRelations + : ) + 6.060237901498659E-4)


df1: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 168 more fields]

In [ ]:
// 6. Найдите другие интересные инсайты в данных :)

In [ ]:
// (
//     df
//     .select($"date", $"audit_timestamp")
//     .withColumn("audit_hhmm", hour( ($"audit_timestamp"/1000).cast(types.TimestampType))*100 + minute( ($"audit_timestamp"/1000).cast(types.TimestampType) ) )
//     .withColumn("audit_timestamp", ($"audit_timestamp"/1000).cast(types.TimestampType))
//     .show(3, false)
// )

In [ ]:
// import org.apache.spark.mllib.linalg._
// import org.apache.spark.mllib.stat.Statistics
// import org.apache.spark.rdd.RDD

// val sc = spark.sparkContext

// val seriesX: RDD[Double] = sc.parallelize(Array(1, 2, 3, 3, 5))  // a series
// // must have the same number of partitions and cardinality as seriesX
// val seriesY: RDD[Double] = sc.parallelize(Array(11, 22, 33, 33, 555))

// // compute the correlation using Pearson's method. Enter "spearman" for Spearman's method. If a
// // method is not specified, Pearson's method will be used by default.
// val correlation: Double = Statistics.corr(seriesX, seriesY, "pearson")
// println(s"Correlation is: $correlation")

// val data: RDD[Vector] = sc.parallelize(
//   Seq(
//     Vectors.dense(1.0, 10.0, 100.0),
//     Vectors.dense(2.0, 20.0, 200.0),
//     Vectors.dense(5.0, 33.0, 366.0))
// )  // note that each Vector is a row and not a column

// // calculate the correlation matrix using Pearson's method. Use "spearman" for Spearman's method
// // If a method is not specified, Pearson's method will be used by default.
// val correlMatrix: Matrix = Statistics.corr(data, "pearson")
// println(correlMatrix.toString)


In [ ]:
spark.close()